## 필요 라이브러리 호출

In [1]:
## 사용 라이브러리 호출
import pandas as pd
import numpy as np
import random 
from urllib.parse import quote
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from scipy.fftpack import fft
from sklearn.decomposition import PCA

## 모델 사용 라이브러리 
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm.notebook import trange
import statistics
from sklearn.metrics import f1_score, classification_report

## 모델 학습 결과 경로 설정 
import os
os.makedirs('./result', exist_ok=True)

## Cuda 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## 랜덤 시드 설정
def set_seed(seed_val):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

# 시드 설정 
seed_val = 77
set_seed(seed_val)

cuda


## 사용 데이터 컬럼 선택

* tag name 이 많은 경우 tag name을 지정하는 것에 있어서 변수 설정이 다소 유연해짐
* tag name 은 순서대로 불러와짐 

In [2]:
# tag name 출력 함수 
def show_column(URL):
    
    # Tag name 데이터 로드
    df = pd.read_csv(URL)
    
    # List 형식으로 변환
    df = df.values.reshape(-1)
    
    return df.tolist()

In [3]:
## tag name 출력 파라미터 설정
table = 'bearing'

NAME_URL = f'http://127.0.0.1:5654/db/tql/datahub/api/v1/get_tag_names.tql?table={table}'

## tag name list 생성 
name = show_column(NAME_URL)

In [4]:
name

['s1-c1',
 's1-c2',
 's1-c3',
 's1-c4',
 's1-c5',
 's1-c6',
 's1-c7',
 's1-c8',
 's2-c1',
 's2-c2',
 's2-c3',
 's2-c4',
 's3-c1',
 's3-c2',
 's3-c3',
 's3-c4']

## TAG Name format 변환 

* 위의 과정에서 Nasa bearing dataset의 모든 Tag Name 을 확인후 사용할 컬럼만 뽑아서 입력할 파라미터 형태로 변환

In [5]:
# 원하는 tag name 설정
# 여기서 tag name 은 컬럼을 의미
tags = ['s1-c5']

# 리스트의 각 항목을 작은따옴표로 감싸고, 쉼표로 구분
tags_ = ",".join(f"'{tag}'" for tag in tags)

# 사용 tag name 확인
print(tags_)

# 해당 값을 모델의 input shape로 설정 
print(len(tags))

's1-c5'
1


## Nasa Bearing Dataset 로드

* 데이터 로드시 train, validation, test 데이터 셋을 각각 Load

* 예제는 3번쨰 bearing의 이상탐지로 진행 -> 's1-c5' Tag Name 사용

* 나머지 상태 빼고 고장이 아니면 전부 정상으로 labeling 해서 진행

In [6]:
# 각 bearing의 상태를 시간 범위로 설정
B1 ={
    "early" : ["2003-10-22 12:06:24" , "2003-10-23 09:14:13"],
    "suspect" : ["2003-10-23 09:24:13" , "2003-11-08 12:11:44"],
    "normal" : ["2003-11-08 12:21:44" , "2003-11-19 21:06:07"],
    "suspect_1" : ["2003-11-19 21:16:07" , "2003-11-24 20:47:32"],
    "imminent_failure" : ["2003-11-24 20:57:32","2003-11-25 23:39:56"]
}
B2 = {
    "early" : ["2003-10-22 12:06:24" , "2003-11-01 21:41:44"],
    "normal" : ["2003-11-01 21:51:44" , "2003-11-24 01:01:24"],
    "suspect" : ["2003-11-24 01:11:24" , "2003-11-25 10:47:32"],
    "imminient_failure" : ["2003-11-25 10:57:32" , "2003-11-25 23:39:56"]
}

B3 = {
    "early" : ["2003-10-22 12:06:24" , "2003-11-01 21:41:44"],
    "normal" : ["2003-11-01 21:51:44" , "2003-11-22 09:16:56"],
    "suspect" : ["2003-11-22 09:26:56" , "2003-11-25 10:47:32"],
    "Inner_race_failure" : ["2003-11-25 10:57:32" , "2003-11-25 23:39:56"]
}

B4 = {
    "early" : ["2003-10-22 12:06:24" , "2003-10-29 21:39:46"],
    "normal" : ["2003-10-29 21:49:46" , "2003-11-15 05:08:46"],
    "suspect" : ["2003-11-15 05:18:46" , "2003-11-18 19:12:30"],
    "Rolling_element_failure" : ["2003-11-19 09:06:09" , "2003-11-22 17:36:56"],
    "Stage_two_failure" : ["2003-11-22 17:46:56" , "2003-11-25 23:39:56"]
}

In [7]:
# 데이터 로드 파라미터 설정

# tag table 이름 설정
table = 'bearing'
# tag name 설정
name = quote(tags_, safe=":/")
# 시간 포멧 설정 
timeformat = quote('2006-01-02 15:04:05.000000')
# Train , validation , test 데이터 셋 설정
# 학습 데이터 시작 시간 설정
start_time = quote('2003-10-22 12:06:24')
# 학습 데이터  끝 시간 설정
end_time = quote('2003-11-25 23:39:56')

In [8]:
# 데이터 로드 함수
# 데이터 로드후 초당 데이터 개수를 판단후 부족한 데이터는 제거
# 이후 같은 초의 데이터를 한줄로 만듬 
def data_load(table, name, start_time, end_time, timeformat):
    
    # 데이터 로드 
    df = pd.read_csv(f'http://127.0.0.1:5654/db/tql/datahub/api/v1/select-rawdata.tql?table={table}&name={name}&start={start_time}&end={end_time}&timeformat={timeformat}')
    
    # 같은 시간대 별 데이터로 전환
    df = df.pivot_table(index='TIME', columns='NAME', values='VALUE', aggfunc='first').reset_index()
    
    # time 설정
    df['TIME'] = pd.to_datetime(df['TIME'], format='%Y-%m-%d %H:%M:%S.%f')
    
    # 초 단위로 그룹화하여 데이터 개수 세기
    df_counts = df.groupby(df['TIME'].dt.floor('S')).size().reset_index(name='count')

    # 데이터 개수가 동일한 그룹만 필터링
    # count 값이 가장 많이 나타나는 값들을 선택
    most_common_count = df_counts['count'].mode()[0]

    # 가장 많이 나타나는 count 값으로 필터링
    filtered_df_counts = df_counts[df_counts['count'] == most_common_count]

    # 필터링된 시간값들을 리스트로 변환
    filtered_times = filtered_df_counts['TIME'].tolist()

    # 원본 데이터프레임에서 필터링된 시간값들만 선택
    filtered_data = df[df['TIME'].dt.floor('S').isin(filtered_times)]

    # TIME을 기준으로 그룹화
    # 초 단위로 반올림
    filtered_data_ = filtered_data.copy()
    filtered_data_.loc[:, 'TIME'] = filtered_data_['TIME'].dt.floor('S')
    grouped = filtered_data_.groupby('TIME')['s1-c5'].apply(list).reset_index()

    # 리스트를 개별 열로 나누기
    s1_c5_df = pd.DataFrame(grouped['s1-c5'].tolist())

    # 'TIME' 열과 병합
    result_df = pd.concat([grouped[['TIME']], s1_c5_df], axis=1)
    
    # label 설정
    # 각 채널데이터 비정상부분의 시간 값을 기준으로 label 설정 
    result_df['label'] = np.where((result_df['TIME'] >= "2003-11-25 10:57:32") & (result_df['TIME'] <= "2003-11-25 23:39:56"), 1, 0)
    
    result_df = result_df.drop(['TIME'], axis=1)

    return result_df

In [9]:
# 데이터 로드

df = data_load(table, name, start_time, end_time, timeformat)
df

,0,1,2,3,4,5,6,7,8,9,...,20471,20472,20473,20474,20475,20476,20477,20478,20479,label
0,-0.105,-0.049,-0.005,-0.100,-0.151,0.046,-0.132,-0.164,-0.110,-0.100,...,-0.088,-0.107,-0.078,-0.093,-0.200,-0.159,-0.237,-0.027,-0.002,0
1,-0.083,-0.132,-0.081,-0.022,-0.129,-0.110,-0.149,-0.168,-0.225,-0.217,...,-0.129,-0.149,-0.046,0.007,-0.132,-0.073,0.056,-0.186,-0.049,0
2,-0.122,-0.244,-0.156,-0.076,0.046,-0.098,-0.142,0.083,-0.195,-0.088,...,-0.107,-0.061,-0.090,-0.049,-0.125,-0.056,-0.137,-0.247,-0.229,0
3,-0.210,-0.125,-0.090,-0.215,-0.225,-0.139,-0.042,-0.090,-0.142,-0.232,...,-0.046,-0.195,-0.085,-0.112,-0.049,-0.146,-0.154,-0.220,-0.090,0
4,-0.088,-0.088,-0.110,-0.269,0.024,-0.054,-0.156,-0.205,-0.056,-0.132,...,0.044,-0.122,-0.115,-0.056,-0.078,-0.002,-0.342,-0.173,-0.161,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2150,-0.212,-0.095,0.044,0.005,-0.364,-0.239,-0.098,-0.066,-0.081,-0.164,...,-0.574,0.127,0.767,0.173,0.217,-0.271,0.085,0.364,-0.166,1
2151,0.432,0.217,-0.627,-0.308,0.073,0.107,-0.359,0.056,-0.159,-0.181,...,-0.161,-0.046,-0.098,-0.186,-0.251,-0.032,-0.017,-0.073,-0.437,1
2152,-0.112,-0.510,0.037,0.063,-0.115,-0.066,0.117,0.405,-0.374,-0.339,...,-0.188,-0.002,-0.085,-0.149,-0.195,-0.227,-0.095,-0.115,-0.293,1
2153,-0.593,0.366,-0.393,-0.312,-0.156,0.288,0.122,-0.105,0.242,-0.166,...,0.510,-0.776,-0.483,0.288,0.359,0.120,-0.173,-0.400,0.505,1


In [10]:
# 학습, 검증, 테스트 데이터 분리

train, test = train_test_split(df, test_size=0.2, shuffle=False)
valid, test = train_test_split(test, test_size=0.5, shuffle=False)

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

## 데이터 전처리

* 각 데이터별로 hanning window, FFT, MinMax Scaling 적용

## hanning window 함수 설정

In [11]:
# hanning window 함수 설정 
def set_hanning_window(sample_rate, df):
    
    # Hanning 윈도우 생성
    hanning_window = np.hanning(sample_rate)

    # 각 행에 Hanning 윈도우 적용
    df_windowed = df.multiply(hanning_window, axis=1)
    
    return df_windowed

In [12]:
# 파라미터 설정
window_length = len(df.columns[:-1])

train_ = set_hanning_window(window_length, train.iloc[:,:-1])
valid_ = set_hanning_window(window_length, valid.iloc[:,:-1])
test_ = set_hanning_window(window_length, test.iloc[:,:-1])

## FFT 함수 설정

In [13]:
# FFT 변환 함수
def change_fft(sample_rate, df):
    # 신호의 총 샘플 수
    N = sample_rate
    
    # 각 행에 대해 FFT 적용
    fft_results = np.zeros((df.shape[0], N // 2 + 1), dtype=float)
    
    for i in range(df.shape[0]):
        # 각 행의 FFT 계산
        yf = fft(df.iloc[i].values)
        
        # FFT 결과의 절댓값을 계산하고 정규화 (유의미한 부분만)
        fft_results[i] = 2.0 / N * np.abs(yf[:N // 2 + 1])
    
    # FFT 결과를 데이터 프레임으로 변환
    fft_df = pd.DataFrame(fft_results)
    
    return fft_df

In [14]:
# 샘플링 주기 -> 초당 데이터 개수 
sampling_rate = len(df.columns[:-1])

# FFT 변환
train_ = change_fft(sampling_rate, train_)
valid_ = change_fft(sampling_rate, valid_)
test_ = change_fft(sampling_rate, test_)

In [15]:
# 스케일러 설정
scaler = MinMaxScaler()

# 스케일러 적용
train_ = scaler.fit_transform(train_.values)
valid_ = scaler.transform(valid_.values)
test_ = scaler.transform(test_.values)

# 데이터 프레임 설정
train_scaled = pd.DataFrame(train_)
valid_scaled = pd.DataFrame(valid_)
test_scaled = pd.DataFrame(test_)

In [16]:
## PCA 적용
# 95%의 분산을 설명하는 주성분 선택
pca = PCA(n_components=0.95)

# 변환 
train_scaled_ = pca.fit_transform(train_scaled)
valid_scaled_ = pca.transform(valid_scaled)
test_scaled_ = pca.transform(test_scaled)

# 데이터 프레임 설정
train_scaled_ = pd.DataFrame(train_scaled_)
valid_scaled_ = pd.DataFrame(valid_scaled_)
test_scaled_ = pd.DataFrame(test_scaled_)

# label 추가
train_scaled_['label'] = train['label'].values
valid_scaled_['label'] = valid['label'].values
test_scaled_['label'] = test['label'].values

print(train_scaled_['label'].value_counts())
print(valid_scaled_['label'].value_counts())
print(test_scaled_['label'].value_counts())

label
0    1724
Name: count, dtype: int64
label
0    215
Name: count, dtype: int64
label
0    181
1     35
Name: count, dtype: int64


## 윈도우 데이터 셋 설정

시계열 데이터 학습을 위해서는 윈도우 사이즈와 슬라이딩 값을 설정해야함

* window size : 몇개의 시간으로 묶을지에 대한 설정
* step size : window 가 이동하는 시간 간격

In [17]:
# 슬라이딩 윈도우 데이터셋 설정 
class SlidingWindowDataset(Dataset):
    def __init__(self, data, window_size, step_size):
        self.data = data
        self.window_size = window_size
        self.step_size = step_size
        self.windows, self.labels = self._create_windows()
    
    # 슬라이딩 윈도우 설정
    def _create_windows(self):
        windows = []
        labels = []
        for i in range(0, len(self.data) - self.window_size + 1, self.step_size):
            window = self.data.iloc[i:i + self.window_size, :-1].values  # 마지막 컬럼 제외
            label_array = self.data.iloc[i:i + self.window_size, -1].values  # 마지막 컬럼이 label
            
            # 레이블 배열에서 하나라도 비정상 값이 있으면 레이블을 1로 설정
            if (label_array == 1).any():
                label = 1  
            else:
                label = 0
                
            windows.append(torch.Tensor(window))
            labels.append(torch.Tensor([label]))  # label을 Tensor로 변환
        return windows, labels
    
    def __len__(self):
        return len(self.windows)
    
    def __getitem__(self, idx):
        return self.windows[idx], self.labels[idx]

In [18]:
# 슬라이딩 윈도우 설정
window_size = 3
step_size = 1 

# 데이터 셋 설정 
train_ = SlidingWindowDataset(train_scaled_, window_size, step_size)
valid_ = SlidingWindowDataset(valid_scaled_, window_size, step_size)
test_ = SlidingWindowDataset(test_scaled_, window_size, step_size)

# 데이터 로더 설정
train_dataloader = DataLoader(train_, batch_size=32, shuffle=False)
valid_dataloader = DataLoader(valid_, batch_size=1, shuffle=False)
test_dataloader = DataLoader(test_, batch_size=1, shuffle=False)

In [19]:
print(list(train_dataloader)[0][0].shape)

torch.Size([32, 3, 1487])


## 학습 모델 설정 

* LSTM AE 기본 모델 사용

In [20]:
# LSTM Autoencoder 클래스 정의
class LSTMAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(LSTMAutoencoder, self).__init__()
        
        # 인코더 LSTM
        self.encoder_lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.encoder_fc = nn.Linear(hidden_dim, 2*hidden_dim)
        
        # 디코더 LSTM
        self.decoder_fc = nn.Linear(2*hidden_dim, hidden_dim)
        self.decoder_lstm = nn.LSTM(hidden_dim, input_dim, num_layers, batch_first=True)

    def forward(self, x):
        # 인코더 부분
        _, (h, _) = self.encoder_lstm(x)
        latent = self.encoder_fc(h[-1])
        
        # 디코더 부분
        hidden = self.decoder_fc(latent).unsqueeze(0).repeat(x.size(1), 1, 1).permute(1, 0, 2)
        output, _ = self.decoder_lstm(hidden)
        
        return output

In [21]:
# 모델 설정 파라미터

# 입력 데이터 컬럼 수
# PCA 한값 
# print(list(train_dataloader)[0][0].shape) 에서 마지막 숫자 
input_dim = 1487

# LSMT hidden state 크기
hidden_dim = 256

# layer 수
num_layers = 3

# 학습률 
learning_rate = 0.01

# 모델 초기화
model = LSTMAutoencoder(input_dim, hidden_dim, num_layers).to(device)

# 손실 함수 및 옵티마이저 설정
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 모델 구조 확인
print(model)

LSTMAutoencoder(
  (encoder_lstm): LSTM(1487, 256, num_layers=3, batch_first=True)
  (encoder_fc): Linear(in_features=256, out_features=512, bias=True)
  (decoder_fc): Linear(in_features=512, out_features=256, bias=True)
  (decoder_lstm): LSTM(256, 1487, num_layers=3, batch_first=True)
)


## 모델 학습 설정

* 학습 중 Loss 값이 제일 높은 모델을 저장 

In [22]:
train_loss = []
total_step = len(train_dataloader)
epoch_in = trange(10, desc='training')
best_Loss= np.inf

for epoch in epoch_in:
    model.to(device)
    model.train()
    running_loss = 0.0

    preds_ = []
    targets_ = []

    for batch_idx, train_data in enumerate(train_dataloader):

        inputs = train_data[0].to(device).float()

        optimizer.zero_grad()

        outputs = model(inputs)
 
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    train_loss.append(running_loss/total_step)
    print(f'\ntrain loss: {np.mean(train_loss)}')

    
    if best_Loss > np.mean(train_loss):
        best_Loss = np.mean(train_loss)
        torch.save(model, f'./result/Nasa_Bearing_LSTM_AE_General2_비지도.pt')
        print('모델 저장')
    epoch_in.set_postfix_str(f"epoch = {epoch}, best_Loss = {best_Loss}")

training:   0%|          | 0/10 [00:00<?, ?it/s]


train loss: 0.1874480980137984
모델 저장

train loss: 0.18224455574872317
모델 저장

train loss: 0.18051004501772516
모델 저장

train loss: 0.17964278979019987
모델 저장

train loss: 0.17912243715039006
모델 저장

train loss: 0.17877553511456945
모델 저장

train loss: 0.17852774798554719
모델 저장

train loss: 0.17834190784574105
모델 저장

train loss: 0.17819736530015498
모델 저장

train loss: 0.1780817310705229
모델 저장


## 임계값 설정

* validation data를 사용하여 임계값 계산 
  * Max + K x 표준편차

In [23]:
# 베스트 모델 로드
model_ = torch.load(f'./result/Nasa_Bearing_LSTM_AE_General2_비지도.pt') 

In [24]:
# validation data 재구성 Loss 계산 
valid_loss = []
with torch.no_grad():
    
    for batch_idx, valid_data in enumerate(valid_dataloader):

        inputs_val = valid_data[0].to(device).float()

        outputs_val = model_(inputs_val)
        loss = criterion(outputs_val, inputs_val)
        
        valid_loss.append(loss.item())
        
# 임계값 설정
# 임계값은 본인이 세운 기준으로 맞추는 것
# 해당 임계값은 Max + K x 표준편차 로 구성, validation 기준으로 k 값을 조정 
threshold =  max(valid_loss) + 10*statistics.stdev(valid_loss)

print(threshold)

0.18937314544431605


## 모델 테스트

* 이전 단계에서 계산한 임계값을 기준으로 테스트 데이터를 통한 모델 테스트 진행

In [25]:
# test 데이터 모델 적용 
test_loss = []
test_label = []
with torch.no_grad():
    
    for batch_idx, test_data in enumerate(test_dataloader):

        inputs_test = test_data[0].to(device).float()
        label = test_data[1].to(device).long()

        outputs_test = model_(inputs_test)
        loss = criterion(outputs_test, inputs_test)
        
        test_loss.append(loss.item())
        test_label.append(label.item())

# 테스트 결과 데이터 프레임 생성
result = pd.DataFrame(test_loss, columns=['Reconst_Loss'])

# 실제 label 설정 
result['label'] = test_label

# 각 임계값 기준 정상, 비정상 구분
result['pred'] = np.where(result['Reconst_Loss']>threshold,1,0)

# 성능 평가

* F1 Score 기준으로 평가 진행 

In [26]:
# Max + 10 x 표준편차 임계값 test 기준 F1 Score 출력 
print(classification_report(result['label'], result['pred']))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       179
           1       1.00      1.00      1.00        35

    accuracy                           1.00       214
   macro avg       1.00      1.00      1.00       214
weighted avg       1.00      1.00      1.00       214



## 과적합 체크

* test 데이터뿐만 아니라 train, validation 데이터를 활용해 F1 score 계산
* 만약 train, validation 결과가 좋지 않다면 모델 과적합으로 판단

In [27]:
# test 데이터 모델 적용 
train_loss = []
train_label = []
train_dataloader = DataLoader(train_, batch_size=1, shuffle=False)

with torch.no_grad():
    
    for batch_idx, train_data in enumerate(train_dataloader):

        inputs_train = train_data[0].to(device).float()
        label = train_data[1].to(device).long()

        outputs_train = model_(inputs_train)
        loss = criterion(outputs_train, inputs_train)
        
        train_loss.append(loss.item())
        train_label.append(label.item())

# 테스트 결과 데이터 프레임 생성
result = pd.DataFrame(train_loss, columns=['Reconst_Loss'])

# 실제 label 설정 
result['label'] = train_label

# 각 임계값 기준 정상, 비정상 구분
result['pred'] = np.where(result['Reconst_Loss']>threshold,1,0)

In [29]:
# Max + 10 x 표준편차 임계값 train 기준 F1 Score 출력 
print(classification_report(result['label'], result['pred'],labels=[0]))

              precision    recall  f1-score   support

           0       1.00      0.83      0.90      1722

   micro avg       1.00      0.83      0.90      1722
   macro avg       1.00      0.83      0.90      1722
weighted avg       1.00      0.83      0.90      1722



In [30]:
# test 데이터 모델 적용 
val_loss = []
val_label = []
with torch.no_grad():
    
    for batch_idx, valid_data in enumerate(valid_dataloader):

        inputs_val = valid_data[0].to(device).float()
        label = valid_data[1].to(device).long()

        outputs_val = model_(inputs_val)
        loss = criterion(outputs_val, inputs_val)
        
        val_loss.append(loss.item())
        val_label.append(label.item())

# 테스트 결과 데이터 프레임 생성
result = pd.DataFrame(val_loss, columns=['Reconst_Loss'])

# 실제 label 설정 
result['label'] = val_label

# 각 임계값 기준 정상, 비정상 구분
result['pred'] = np.where(result['Reconst_Loss']>threshold,1,0)

In [31]:
# Max + 10 x 표준편차 임계값 validation 기준 F1 Score 출력 
print(classification_report(result['label'], result['pred'],labels=[0]))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       213

    accuracy                           1.00       213
   macro avg       1.00      1.00      1.00       213
weighted avg       1.00      1.00      1.00       213

